In [1]:
# Question 5

import gzip
from collections import defaultdict

def readGz(f):
  for l in gzip.open(f):
    yield eval(l)

In [2]:
# establish training set and validation set

train_set = set()
validation_set = set()
count = 0

for l in readGz("train.json.gz"):    
    count += 1
    if count <= 100000:
        train_set.add(l['reviewerID'] + '-' + l['itemID'])
    else:
        validation_set.add(l['reviewerID'] + '-' + l['itemID'] )

In [3]:
allRatings = []
userRatings = defaultdict(list)
count = 0

for l in readGz("train.json.gz"):
    user,business = l['reviewerID'],l['itemID']
    allRatings.append(l['rating'])
    userRatings[user].append(l['rating'])
    count += 1
    if count == 100000:
        break
    
globalAverage = sum(allRatings) / len(allRatings)

print("The value of alpha is " + str(globalAverage))

The value of alpha is 4.232


In [4]:
# obtain the ratings of the validation set
allRatings = []

for l in readGz("train.json.gz"):
    allRatings.append(l['rating'])

validation_ratings = allRatings[100000:]
len(validation_ratings)

100000

In [5]:
# MSE on the validation set
import numpy as np

validation_ratings = np.array(validation_ratings)

MSE = sum((validation_ratings-globalAverage)**2)/len(validation_ratings)

print("The MSE of the trivial predictor on the validation set is " + str(MSE))

The MSE of the trivial predictor on the validation set is 1.222481119999121


In [6]:
# Question 6

# construct a rating dictionary for the training set
TR_dict = defaultdict(float)
train_ratings = allRatings[0:100000]
count = 0

for l in train_set:
    TR_dict[l] = train_ratings[count]
    count += 1

In [7]:
# construct initial beta_u and beta_i dictionary
beta_u = defaultdict(float)
beta_i = defaultdict(float)
lamda = 1
alpha = globalAverage

# construct user_ratings dictionary and item_ratings dictionary
UR_dict = defaultdict(list)
IR_dict = defaultdict(list)

for key in TR_dict:
    UR_dict[key.split('-')[0]].append(TR_dict[key])
    IR_dict[key.split('-')[1]].append(TR_dict[key])


In [8]:
# construct user_average dictionary and item_average dictionary
UA_dict = defaultdict(float)
IA_dict = defaultdict(float)

for key in UR_dict:
    UA_dict[key] = sum(UR_dict[key])/len(UR_dict[key])

In [9]:
for key in IR_dict:
    IA_dict[key] = sum(IR_dict[key])/len(IR_dict[key])

In [36]:
# initialize beta_u and beta_i
for key in UA_dict:
    beta_u[key] = UA_dict[key] - alpha

for key in IA_dict:
    beta_i[key] = IA_dict[key] - alpha
k =0
for i in beta_i:
    k+= beta_i[i]
k

-81.75614914973565

In [11]:
# iterate the procedure until convergence
# First of all, we have to construct user-item and item-user dictionaries
UI_dict = defaultdict(list)
IU_dict = defaultdict(list)

for l in train_set:
    UI_dict[l.split('-')[0]].append(l.split('-')[1])
    IU_dict[l.split('-')[1]].append(l.split('-')[0])

In [29]:
beta_u_new = beta_u.copy()
beta_i_new = beta_i.copy()
UR_dict

36427

In [37]:
# iteration
alpha_new = alpha
lamda = 1


for i in range(100):
    # update alpha
    sum_1 = 0
    sum_2 = 0
    sum_3 = 0
    for key in TR_dict:
        sum_1 += TR_dict[key]
    for key in beta_u_new:
        sum_2 += beta_u_new[key]
    for key in beta_i_new:
        sum_3 += beta_i_new[key]
    alpha_new = (sum_1 - sum_2 -sum_3)/len(train_set)
    print(sum_2)
    print(sum_3)
    # update beta_u
    sum_1 = 0
    sum_2 = 0
    sum_3 = 0

    for key in beta_u_new:
        count_item = []
        sum_1 = sum(UR_dict[key])
        sum_2 = alpha_new*len(UR_dict[key])
        for item in UI_dict[key]:
            count_item.append(item)
        for x in count_item:
            sum_3 += beta_i_new[x]
        beta_u_new[key] = (sum_1 - sum_2 -sum_3)/(lamda + len(UI_dict[key]))
        sum_1 = 0
        sum_2 = 0
        sum_3 = 0

    # update beta_i
    sum_1 = 0
    sum_2 = 0
    sum_3 = 0

    for key in beta_i_new:
        count_user = []
        sum_1 = sum(IR_dict[key])
        sum_2 = alpha_new*len(IR_dict[key])
        for user in IU_dict[key]:
            count_user.append(user)
        for x in count_user:
            sum_3 += beta_u_new[x]
        beta_i_new[key] = (sum_1 - sum_2 - sum_3)/(lamda + len(IU_dict[key]))
        sum_1 = 0
        sum_2 = 0
        sum_3 = 0       
    # calculate MSE on the validation set
    rating_prediction = []
    validation_user = []
    validation_item = []

    for l in validation_set:
        validation_user.append(l.split('-')[0])
        validation_item.append(l.split('-')[1])   

-46.62368735521407
-46.623687355215395


In [32]:
for i in range(len(validation_user)):
    if validation_user[i] in beta_u_new and validation_item[i] in beta_i_new:
        prediction = alpha_new + beta_u_new[validation_user[i]] + beta_i_new[validation_item[i]]
        rating_prediction.append(prediction)
    elif validation_user[i] not in beta_u_new and validation_item[i] in beta_i_new:
        prediction = alpha_new + beta_i_new[validation_item[i]]
        rating_prediction.append(prediction)
    elif validation_user[i] in beta_u_new and validation_item[i] not in beta_i_new:
        prediction = alpha_new + beta_u_new[validation_item[i]]
        rating_prediction.append(prediction)
    else:
        prediction = alpha_new
        rating_prediction.append(prediction)

rating_prediction = np.array(rating_prediction)
print(len(rating_prediction))
validation_MSE = sum((rating_prediction - validation_ratings)**2)/len(rating_prediction)
print("The MSE on the validation set is " + str(validation_MSE))

100000
The MSE on the validation set is 1.5909165487246155


In [16]:
# Question 7
max_user = []
max_item = []
min_user = []
min_item = []

for key in beta_u_new:
    max_user.append(beta_u_new[key])
max_user_beta = max(max_user)
max_user_ID = [k for k,v in beta_u_new.items() if v == max_user_beta][0]
print("The user with the largest value of beta is " + str(max_user_ID))

for key in beta_i_new:
    max_item.append(beta_i_new[key])
max_item_beta = max(max_item)
max_item_ID = [k for k,v in beta_i_new.items() if v == max_item_beta][0]
print("The item with the largest value of beta is " + str(max_item_ID))

for key in beta_u_new:
    min_user.append(beta_u_new[key])
min_user_beta = min(min_user)
min_user_ID = [k for k,v in beta_u_new.items() if v == min_user_beta][0]
print("The user with the smallest value of beta is " + str(min_user_ID))

for key in beta_i_new:
    min_item.append(beta_i_new[key])
min_item_beta = min(min_item)
min_item_ID = [k for k,v in beta_i_new.items() if v == min_item_beta][0]
print("The item with the smallest value of beta is " + str(min_item_ID))

The user with the largest value of beta is U157710283
The item with the largest value of beta is I531372057
The user with the smallest value of beta is U691931970
The item with the smallest value of beta is I902264614


In [18]:
# Question 8

# After several trials, we discovered that when lamda equals 100, the MSE on the validation set 
# is smallest.

# iteration
alpha_new = alpha
lamda = 100
beta_u_new = beta_u.copy()
beta_i_new = beta_i.copy()

for i in range(100):
    # update alpha
    sum_1 = 0
    sum_2 = 0
    sum_3 = 0
    for key in TR_dict:
        sum_1 += TR_dict[key]
    for key in beta_u_new:
        sum_2 += beta_u_new[key]
    for key in beta_i_new:
        sum_3 += beta_i_new[key]
    alpha_new = (sum_1 - sum_2 -sum_3)/len(train_set)

    # update beta_u
    sum_1 = 0
    sum_2 = 0
    sum_3 = 0

    for key in beta_u_new:
        count_item = []
        sum_1 = sum(UR_dict[key])
        sum_2 = alpha_new*len(UR_dict[key])
        for item in UI_dict[key]:
            count_item.append(item)
        for x in count_item:
            sum_3 += beta_i_new[x]
        beta_u_new[key] = (sum_1 - sum_2 -sum_3)/(lamda + len(UI_dict[key]))
        sum_1 = 0
        sum_2 = 0
        sum_3 = 0

    # update beta_i
    sum_1 = 0
    sum_2 = 0
    sum_3 = 0

    for key in beta_i_new:
        count_user = []
        sum_1 = sum(IR_dict[key])
        sum_2 = alpha_new*len(IR_dict[key])
        for user in IU_dict[key]:
            count_user.append(user)
        for x in count_user:
            sum_3 += beta_u_new[x]
        beta_i_new[key] = (sum_1 - sum_2 - sum_3)/(lamda + len(IU_dict[key]))
        sum_1 = 0
        sum_2 = 0
        sum_3 = 0 
        
# calculate MSE on the validation set
rating_prediction = []
validation_user = []
validation_item = []

for l in validation_set:
    validation_user.append(l.split('-')[0])
    validation_item.append(l.split('-')[1])

for i in range(len(validation_user)):
    if validation_user[i] in beta_u_new and validation_item[i] in beta_i_new:
        prediction = alpha_new + beta_u_new[validation_user[i]] + beta_i_new[validation_item[i]]
        rating_prediction.append(prediction)
    elif validation_user[i] not in beta_u_new and validation_item[i] in beta_i_new:
        prediction = alpha_new + beta_i_new[validation_item[i]]
        rating_prediction.append(prediction)
    elif validation_user[i] in beta_u_new and validation_item[i] not in beta_i_new:
        prediction = alpha_new + beta_u_new[validation_item[i]]
        rating_prediction.append(prediction)
    else:
        prediction = alpha_new
        rating_prediction.append(prediction)

rating_prediction = np.array(rating_prediction)

validation_MSE = sum((rating_prediction - validation_ratings)**2)/len(rating_prediction)
print("The MSE on the validation set is " + str(validation_MSE))

The MSE on the validation set is 1.2240486232479748


In [23]:
# make predictions for the test set

predictions = open("predictions_Rating.txt", 'w')
for l in open("pairs_Rating.txt"):
    if l.startswith("reviewerID"):
        #header
        predictions.write(l)
        continue
    u,i = l.strip().split('-')
    if u in beta_u_new and i in beta_i_new:
        predictions.write(u + '-' + i + ',' + str(alpha_new + beta_u_new[u] + beta_i_new[i]) + '\n')
    elif u not in beta_u_new and i in beta_i_new:
        predictions.write(u + '-' + i + ',' + str(alpha_new + beta_i_new[i]) + '\n')
    elif u in beta_u_new and i not in beta_i_new:
        predictions.write(u + '-' + i + ',' + str(alpha_new + beta_u_new[i]) + '\n')
    else:
        predictions.write(u + '-' + i + ',' + str(alpha_new) + '\n')
          
predictions.close()